<a href="https://colab.research.google.com/github/UFM-Market-Trends/UFM-Market-Trends-SDK/blob/main/examples/2_Agregados_Monetarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install pyxlsb
!pip install fpdf
import datetime
!pip install git+https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git
import ufmtrends_sdk as trends
today = datetime.date.today().strftime("%Y-%m-%d")
# !pip install xlrd
!pip install --upgrade xlrd

  Using cached pyxlsb-1.0.10-py2.py3-none-any.whl (23 kB)
  Using cached fpdf-1.7.2-py2.py3-none-any.whl
  Cloning https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to /tmp/pip-req-build-47rswdqo
  Running command git clone --filter=blob:none --quiet https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git /tmp/pip-req-build-47rswdqo
  Resolved https://github.com/UFM-Market-Trends/UFM-Market-Trends-SDK.git to commit 9064e503fcb23b4a44b532364ce5844c318ab5f7
  Preparing metadata (setup.py) ... done
  Created wheel for ufmtrends-sdk: filename=ufmtrends_sdk-0.1.7-py3-none-any.whl size=13195 sha256=e97b14b2411b3a5e0f945debd47b3e00eb4466fbe37be4376aa66b93aad34484
  Stored in directory: /tmp/pip-ephem-wheel-cache-403erndc/wheels/d8/63/72/fd94fcb7285b31cd0125721e8d1340b8842b4efc3b6d212027
Successfully built ufmtrends-sdk
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


# Agregados Monetarios

https://www.banguat.gob.gt/page/iii-base-monetaria-y-agregados-monetarios

## Base monetaria

Base monetaria restringida y la base monetaria amplia.

https://www.banguat.gob.gt/es/page/iii1-base-monetaria

In [2]:
monetary_base_data_url = "https://www.banguat.gob.gt/sites/default/files/banguat/estamonfin/estamonfin031.xls"

RESTRICTED_MONETARY_BASE = "Base monetaria restringida"
AMPLE_MONETARY_BASE = "Base monetaria amplia"
def download_monetary_base(monetary_base_url):
  '''
  '''
  monetary_base_df = pd.read_excel(monetary_base_url,
                                  skiprows=10,
                                  skipfooter=3)

  monetary_base_df = monetary_base_df.iloc[:,[2, 6, 16]]


  monetary_base_df.columns = ['date', RESTRICTED_MONETARY_BASE, 
                              AMPLE_MONETARY_BASE]

  start_year = 2001

  start_date = datetime.datetime(year=start_year, 
                                month=12, 
                                day=1)

  months_since_then = len(monetary_base_df["date"]) # trimestres hasta el presente
  months = pd.period_range(start_date, periods=months_since_then,freq='M')

  dates = pd.PeriodIndex(months, freq='M').to_timestamp()

  monetary_base_df["date"] = dates 
  monetary_base_df.insert(loc=1, column="month", value=months, allow_duplicates=True)

  return monetary_base_df


monetary_base_dataframe = download_monetary_base(monetary_base_data_url)
monetary_base_dataframe

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1091)>

## Agregados monetarios

Agregados monetarios M1, M2, y M3.

* https://www.banguat.gob.gt/es/page/iii2-agregados-monetarios

In [ ]:
money_supply_data_url = "https://www.banguat.gob.gt/sites/default/files/banguat/estamonfin/estamonfin032.xls"



def download_money_supply(money_supply_url):
  '''
  '''

  money_supply_df = pd.read_excel(money_supply_url,
                                  skiprows=10,
                                  skipfooter=6)
  
  money_supply_df = money_supply_df.iloc[:,[2,5,8,12]]

  money_supply_df.columns = ['date', 'M1', 'M2', 'M3']

  start_year = 2001

  start_date = datetime.datetime(year=start_year, 
                                month=12, 
                                day=1)

  months_since_then = len(money_supply_df["date"])
  months = pd.period_range(start_date, periods=months_since_then,freq='M')

  dates = pd.PeriodIndex(months, freq='M').to_timestamp()

  money_supply_df["date"] = dates 
  money_supply_df.insert(loc=1, column="month", value=months, allow_duplicates=True)

  return money_supply_df

money_supply_dataframe = download_money_supply(money_supply_data_url)
money_supply_dataframe

In [ ]:
last_many_months = -12 * 5 + 1

RESTRICTED_BASE_GROWTH = "Crecimiento base monetaria restringida"
M1_SUPPLY_GROWTH = "Crecimiento M1"
AMPLE_BASE_GROWTH = "Crecimiento base monetaria amplia"
M3_SUPPLY_GROWTH = "Crecimiento M3"


money_supply_growth_data = {
    'labels': monetary_base_dataframe['month'][last_many_months:].astype(str),
    'dates': monetary_base_dataframe['date'][last_many_months:],
    RESTRICTED_BASE_GROWTH: trends.get_yearonyear_variation(list(monetary_base_dataframe[RESTRICTED_MONETARY_BASE]),
                                                       inter_values_size=12)[last_many_months:],
    M1_SUPPLY_GROWTH: trends.get_yearonyear_variation(list(money_supply_dataframe['M1']),
                         inter_values_size=12)[last_many_months:],

    #
    AMPLE_BASE_GROWTH: trends.get_yearonyear_variation(list(monetary_base_dataframe[AMPLE_MONETARY_BASE]),
                                                       inter_values_size=12)[last_many_months:],
    M3_SUPPLY_GROWTH: trends.get_yearonyear_variation(list(money_supply_dataframe['M3']),
                         inter_values_size=12)[last_many_months:],
}

plt.plot(money_supply_growth_data['dates'],
         money_supply_growth_data[RESTRICTED_BASE_GROWTH],
         label="Base monetaria restringida"
         )
plt.plot(money_supply_dataframe['date'][last_many_months:],
         money_supply_growth_data[M1_SUPPLY_GROWTH],
         color="red",
         label='M1'
         )
plt.ylim(0,0.30)
# plt.ylabel("Crecimiento interanual")
plt.title("Agregados monetarios (moneda nacional)")
plt.legend()
plt.show()


#
plt.plot(money_supply_growth_data['dates'],
         money_supply_growth_data[AMPLE_BASE_GROWTH],
         label='Base monetaria amplia'
         )
plt.plot(money_supply_dataframe['date'][last_many_months:],
         money_supply_growth_data[M3_SUPPLY_GROWTH],
         color="red",
         label='M3'
         )
plt.ylim(-0.1,0.30)
plt.ylabel("Crecimiento interanual")
plt.title("Agregados monetarios (incluye dólar)")
plt.legend()
plt.show()


In [ ]:
money_supply_growth_csv = "Agregados-Monetarios-Crecimiento-Interanual.csv"
money_supply_growth_df = pd.DataFrame(money_supply_growth_data)
money_supply_growth_df.iloc[:,2:] *= 100
money_supply_growth_df.to_csv(money_supply_growth_csv, index=False)

In [ ]:
# # https://colab.research.google.com/notebooks/snippets/gcs.ipynb
# from google.colab import auth
# auth.authenticate_user()

# Vertex AI authenticates automatically

# project_id = 'ufm-market-trends'
# !gcloud config set project {project_id}

In [ ]:
bucket_path = 'ufm-market-trends/guatemala/agregados-monetarios/agregados-monetarios-crecimiento'
csv_file = money_supply_growth_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

## Medios de pago
* M2 moneda nacional http://www.banguat.gob.gt/es/page/en-moneda-nacional-3 (imm18a.xls)
* M2 moneda extranjera http://www.banguat.gob.gt/page/en-moneda-extranjera-0 (imm18b.xls)
* Base monetaria http://www.banguat.gob.gt/page/base-monetaria (imm14.xls)

In [ ]:
total_monetary_base_url = "http://www.banguat.gob.gt/sites/default/files/banguat/imm/imm14.xls"

TOTAL_MONETARY_BASE = "Base monetaria"
def download_total_monetary_base(url):
  df = pd.read_excel(url,
                     skiprows=5,
                     skipfooter=2)

  df = df.iloc[:, 2:]

  df = df.transpose().stack().reset_index()

  month_rows = np.arange(0,12)
  df = df.drop(month_rows).reset_index(drop=True)
  df.columns = ['date', 'month', TOTAL_MONETARY_BASE] 

  start_year = 1995

  start_date = datetime.datetime(year=start_year, 
                                 month=1,
                                 day=1)


  months_amount = len(df["month"])
  months = pd.period_range(start_date, periods=months_amount, freq='M')

  df["date"] = pd.PeriodIndex(months, freq='M').to_timestamp()
  df["month"] = months

  df[TOTAL_MONETARY_BASE] = df[TOTAL_MONETARY_BASE].astype(str).str.replace(',', '').astype(float)

  return df


m2_national_currency_url = "http://www.banguat.gob.gt/sites/default/files/banguat/imm/imm18a.xls"
M2_NATIONAL_CURRENCY = "M2 Moneda nacional"

def download_M2_national(url):
  df = pd.read_excel(url,
                     skiprows=3,
                     skipfooter=4)
  
  df = df.iloc[:,2:-1]

  df = df.transpose().stack().reset_index()

  month_rows = np.arange(0,12)
  df = df.drop(month_rows).reset_index(drop=True)
  df.columns = ['date', 'month', M2_NATIONAL_CURRENCY] 

  start_year = 1995

  start_date = datetime.datetime(year=start_year, 
                                 month=1,
                                 day=1)


  months_amount = len(df["month"])
  months = pd.period_range(start_date, periods=months_amount, freq='M')

  df["date"] = pd.PeriodIndex(months, freq='M').to_timestamp()
  df["month"] = months


  return df

m2_foreign_currency_url = "http://www.banguat.gob.gt/sites/default/files/banguat/imm/imm18b.xls"
M2_FOREIGN_CURRENCY = "M2 Moneda extranjera"

def download_M2_foreign(url):
  df = pd.read_excel(url,
                     skiprows=3,
                     skipfooter=5)
  
  df = df.iloc[:,2:-1]

  df = df.transpose().stack().reset_index()

  month_rows = np.arange(0,12)
  df = df.drop(month_rows).reset_index(drop=True)
  df.columns = ['date', 'month', M2_FOREIGN_CURRENCY] 

  start_year = 1995

  start_date = datetime.datetime(year=start_year, 
                                 month=1,
                                 day=1)


  months_amount = len(df["month"])
  months = pd.period_range(start_date, periods=months_amount, freq='M')

  df["date"] = pd.PeriodIndex(months, freq='M').to_timestamp()
  df["month"] = months


  return df


total_monetary_base_df = download_total_monetary_base(total_monetary_base_url)
print(total_monetary_base_df)

m2_national_df = download_M2_national(m2_national_currency_url)
print(m2_national_df)

m2_foreign_df = download_M2_foreign(m2_foreign_currency_url)
print(m2_foreign_df)


In [ ]:
last_many_months = -12 * 17 + 1

mobile_avg_m2national = np.array(trends.get_accumulated_values(
    trends.get_yearonyear_variation(list(m2_national_df[M2_NATIONAL_CURRENCY]),inter_values_size=12)[last_many_months:]
    , lot_size=12)[last_many_months+12:])/12

plt.plot(m2_national_df['date'][-len(mobile_avg_m2national):],
         mobile_avg_m2national,
         label="Crecimiento M2", color="r")
plt.ylim(0,0.20)
plt.ylabel("Media móvil 12 meses")
plt.legend()



mobile_avg_totalmoneybase = np.array(trends.get_accumulated_values(
    trends.get_yearonyear_variation(list(total_monetary_base_df[TOTAL_MONETARY_BASE]),
                             inter_values_size=12)[last_many_months:]
    , lot_size=12)[last_many_months+12:])/12

plt.plot(total_monetary_base_df['date'][-len(mobile_avg_totalmoneybase):],
         mobile_avg_totalmoneybase,
         label="Crecimiento base monetaria", color="b")
plt.ylim(0,0.225)
plt.ylabel("Media móvil 12 meses")
plt.legend()
plt.title("Agregados monetarios quetzal")
plt.show()


quetzal_monetary_aggregates_data = {
    "Mes": m2_national_df['month'].astype(str)[-len(mobile_avg_m2national):],
    "Fecha": m2_national_df['date'][-len(mobile_avg_m2national):],
    "Crecimiento M2": mobile_avg_m2national,
    "Crecimiento base monetaria": mobile_avg_totalmoneybase
}

In [ ]:
quetzal_monetary_aggregates_csv = "Agregados-Monetarios-Quetzal-Media-Movil-12-Meses.csv"
quetzal_monetary_aggregates_df = pd.DataFrame(quetzal_monetary_aggregates_data)
quetzal_monetary_aggregates_df.iloc[:,2:] *= 100
quetzal_monetary_aggregates_df.to_csv(quetzal_monetary_aggregates_csv, index=False)

In [ ]:
bucket_path = 'ufm-market-trends/guatemala/agregados-monetarios/agregados-monetarios-crecimiento'
csv_file = quetzal_monetary_aggregates_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}

## Dolarización economía

In [ ]:
last_many_months = -12 * 19

DOLARIZATION_ECONOMY = "Dolarización economía"

dolarization_economy_data = {
    'labels': m2_national_df['month'].astype(str)[last_many_months:],
    'dates': m2_national_df['date'][last_many_months:],
    DOLARIZATION_ECONOMY: list(m2_foreign_df[M2_FOREIGN_CURRENCY][last_many_months:] 
          / 
          m2_national_df[M2_NATIONAL_CURRENCY][last_many_months:])
}

plt.plot(dolarization_economy_data['dates'],
         dolarization_economy_data[DOLARIZATION_ECONOMY])
plt.title("Dolarización economía")
plt.ylabel("M2 en dólares / M2 en quetzales")
plt.show()

In [ ]:
dolarization_economy_csv = "Dolarizacion-Economia.csv"
dolarization_economy_df = pd.DataFrame(dolarization_economy_data)
dolarization_economy_df.iloc[:,2:] *= 100
dolarization_economy_df.to_csv(dolarization_economy_csv, index=False)

In [ ]:
bucket_path = 'ufm-market-trends/guatemala/agregados-monetarios/dolarizacion-economia'
csv_file = dolarization_economy_csv

!gsutil -h "Cache-Control:public, max-age=1" cp {csv_file} gs://{bucket_path}
!gsutil cp gs://{bucket_path}/{csv_file} gs://{bucket_path}/{'archivo'}/{today+'-'+csv_file}  
!gsutil cat gs://{bucket_path}/{csv_file}